# Basic LSTM Model

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential
from tensorflow.keras.optimizers.legacy import Adam
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
import seaborn as sns
import markov
import random
from markov.api.schemas.model_recording import SingleTagInferenceRecord,RecordMetaType,RecordCustomMetric

2023-08-19 02:43:46.950612: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 02:43:46.991536: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-19 02:43:46.992481: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-19 02:43:47.748861: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Data Preprocessing: Loads cleaned tweet data, tokenizes text and labels, and splits into train and test sets.

In [3]:
data = pd.read_csv('../Data/Cleaned_Tweets_stopwords.csv')

text = data['cleaned_text'].tolist()
labels = data['label'].tolist()

In [9]:
max_words = 10000
max_seq_len = 100

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(text)
sequences = tokenizer.texts_to_sequences(text)
word_index = tokenizer.word_index

X = pad_sequences(sequences, maxlen=max_seq_len)
y = np.array(labels)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Word Embeddings: Utilizes pre-trained GloVe word vectors to create an embedding matrix for the words in the dataset.

In [10]:
embedding_dim = 100

embeddings_index = {}
with open('../../../notebook/glove.twitter.27B.100d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [11]:
num_words = min(max_words, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in word_index.items():
    if i >= max_words:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


### LSTM Model: Constructs a Keras Sequential model with an Embedding layer, LSTM layer, and Dense layer for sentiment analysis of tweets, and compiles it with appropriate loss and optimizer.

In [12]:
model = Sequential()
model.add(Embedding(num_words, embedding_dim, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))
model.add(LSTM(128, dropout=0.1, recurrent_dropout=0.2))
model.add(Dense(1, activation='relu'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [13]:

MODEL_NAME = "Basic_LSTM"

markov.keras.auto_record(
    name=MODEL_NAME,
    notes="LSTM Basic Model Using Keras",
    project_id="vXmg4DkwjkU3JR" # team-2 project
)

### Trains a sentiment analysis model, records metrics, and plots training progress.

In [14]:
model.fit(X_train, y_train, batch_size=64, epochs=10, validation_data=(X_val, y_val), verbose=0)

ℹ Creating MarkovML model object Model: Basic_LSTM. Call model_object.register() to register with MarkovML backend.
✔ Creating Model Model: Basic_LSTM⠙ Creating Model Model: Basic_LSTM⠹ Creating Model Model: Basic_LSTM⠸ Creating Model Model: Basic_LSTM⠼ Creating Model Model: Basic_LSTM⠴ Creating Model Model: Basic_LSTM⠦ Creating Model Model: Basic_LSTM⠧ Creating Model Model: Basic_LSTM⠇ Creating Model Model: Basic_LSTM⠏ Creating Model Model: Basic_LSTM⠋ Creating Model Model: Basic_LSTM⠙ Creating Model Model: Basic_LSTM⠹ Creating Model Model: Basic_LSTM⠸ Creating Model Model: Basic_LSTM⠼ Creating Model Model: Basic_LSTM⠴ Creating Model Model: Basic_LSTM⠦ Creating Model Model: Basic_LSTM⠧ Creating Model Model: Basic_LSTM⠇ Creating Model Model: Basic_LSTM⠏ Creating Model Model: Basic_LSTM⠋ Creating Model Model: Basic_LSTM⠙ Creating Model Model: Basic_LSTM⠹ Creating Model Model: Basic_LSTM⠸ Creating Model Model: Basic_LSTM⠼ Creating Model Model: Basic_LSTM⠴ Creating Model Model: Basic_LSTM

 [Elapsed Time: 0:02:33]  [|                      #             |] (  0.2 B/s) 


 [Elapsed Time: 0:02:38]  [|  #                                 |] (  0.6 B/s) 

⠙

✔Please wait while we send the remaining data to the markov backend... Recording has successfully completed.
ℹ You can view the experiment at \https://app.markovml.com/pioneer-wsp-27a2ukk2as/proj/vXmg4DkwjkU3JR/experiments/hp-c5hEdGspVmtdPbwhF94iZk9]8;;\


# Finding the Best Hyper Parameters

In [16]:
def create_lstm_model(learning_rate=0.001, lstm_units=128, dropout_rate=0.2):
    model = Sequential()
    model.add(Embedding(num_words, embedding_dim, input_length=max_seq_len, weights=[embedding_matrix], trainable=False))
    model.add(LSTM(lstm_units, dropout=dropout_rate, recurrent_dropout=dropout_rate))
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

## Hyperparameter Tuning
- Performs hyperparameter tuning using Randomized Search for the LSTM model.
- Identifies the best hyperparameters (learning rate, LSTM units, dropout rate, batch size) based on accuracy.

In [29]:


param_dist = {
    'learning_rate': [0.001],
    'lstm_units': [  256],
    'dropout_rate': [0.3],
    'batch_size': [128]
}

best_score = float('-inf')
best_params = None
for _ in range(10):  # Number of iterations for random search 
    try:
        learning_rate = np.random.choice(param_dist['learning_rate'])
        lstm_units = np.random.choice(param_dist['lstm_units'])
        dropout_rate = np.random.choice(param_dist['dropout_rate'])
        batch_size = np.random.choice(param_dist['batch_size'])

        model = create_lstm_model(learning_rate=learning_rate, lstm_units=lstm_units, dropout_rate=dropout_rate)
        MODEL_NAME = f"LSTM_Parameters-{learning_rate}-{lstm_units}-{dropout_rate}-{batch_size}"

        markov.keras.auto_record(
            name=MODEL_NAME,
            notes=f"LSTM-{learning_rate}-{lstm_units}-{dropout_rate}-{batch_size}",
            project_id="vXmg4DkwjkU3JR" # team-2 project
        )
        model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_val, y_val),verbose=0)
    except Exception as e:
        print("An error occurred:", str(e))


ℹ Creating MarkovML model object Model: LSTM_Parameters-0.001-256-0.3-128. Call model_object.register() to register with MarkovML backend.
⠋ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠙ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠹ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠸ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠼ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠴ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠦ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠧ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128

 [Elapsed Time: 0:17:17]  [|    #                               |] ( 10.4 s/B) 

✔ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠏ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠋ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠙ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠹ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠸ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠼ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠴ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128⠦ Creating Model Model: LSTM_Parameters-0.001-256-0.3-128 Model creation for model Model: LSTM_Parameters-0.001-256-0.3-128 successful!
ℹ Creating ExperimentRecorder object LSTM_Parameters-0.001-256-0.3-128. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Re

 [Elapsed Time: 0:17:19]  [|                        #           |] ( 10.4 s/B) 

⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentRecorder with Markov Backend.⠧ Registering ExperimentRecorder with Markov Backend.

 [Elapsed Time: 0:17:16]  [| #                                  |] ( 34.6 s/B) 

✔ Registering ExperimentRecorder with Markov Backend.⠏ Registering ExperimentRecorder with Markov Backend.⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend.⠦ Registering ExperimentRecorder with Markov Backend.⠧ Registering ExperimentRecorder with Markov Backend.⠇ Registering ExperimentRecorder with Markov Backend.⠏ Registering ExperimentRecorder with Markov Backend.⠋ Registering ExperimentRecorder with Markov Backend.⠙ Registering ExperimentRecorder with Markov Backend.⠹ Registering ExperimentRecorder with Markov Backend.⠸ Registering ExperimentRecorder with Markov Backend.⠼ Registering ExperimentRecorder with Markov Backend.⠴ Registering ExperimentRecorder with Markov Backend. ExperimentRecorder successfully registered.
ℹ

 [Elapsed Time: 0:07:28]  [|                     #              |] ( 15.0 s/B) 
 [Elapsed Time: 0:42:29]  [|           #                        |] ( 25.5 s/B) 

KeyboardInterrupt: 

# Creating model with the best found Parameters

### Creates a model with the best parameters and trains it, visualizing loss and accuracy trends over epochs.

In [8]:
learning_rate=0.01
lstm_units=256
dropout_rate=0.2
batch_size=128
best_model = create_lstm_model(learning_rate=learning_rate,
                               lstm_units=lstm_units,
                               dropout_rate=dropout_rate)
MODEL_NAME = f"Best LSTM Parameters"

markov.keras.auto_record(
    name=MODEL_NAME,
    notes=f"LSTM_Parameters=best",
    project_id="vXmg4DkwjkU3JR" # team-2 project new
)

In [9]:
best_model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_val, y_val) ,verbose=1)

ℹ Creating MarkovML model object Model: Testing Best_LSTM Model. Call model_object.register() to register with MarkovML backend.
✔ Creating Model Model: Testing Best_LSTM Model⠙ Creating Model Model: Testing Best_LSTM Model⠹ Creating Model Model: Testing Best_LSTM Model⠸ Creating Model Model: Testing Best_LSTM Model⠼ Creating Model Model: Testing Best_LSTM Model⠴ Creating Model Model: Testing Best_LSTM Model⠦ Creating Model Model: Testing Best_LSTM Model⠧ Creating Model Model: Testing Best_LSTM Model⠇ Creating Model Model: Testing Best_LSTM Model⠏ Creating Model Model: Testing Best_LSTM Model⠋ Creating Model Model: Testing Best_LSTM Model⠙ Creating Model Model: Testing Best_LSTM Model⠹ Creating Model Model: Testing Best_LSTM Model⠸ Creating Model Model: Testing Best_LSTM Model⠼ Creating Model Model: Testing Best_LSTM Model⠴ Creating Model Model: Testing Best_LSTM Model⠦ Creating Model Model: Testing Best_LSTM Model⠧ Creating Model Model: Testing Best_LSTM Model⠇ Creating Model Model: T

 [Elapsed Time: 0:00:02]  [|                    #               |] ( 49.9 B/s) 

 2/42 [>.............................] - ETA: 21s - loss: 0.6427 - accuracy: 0.6133 

 [Elapsed Time: 0:00:00]  [|#                                   |] (  0.0 s/B) 

 4/42 [=>............................] - ETA: 20s - loss: 1.1463 - accuracy: 0.5918

...

42/42 [==============================] - 29s 673ms/step - loss: 0.0765 - accuracy: 0.9721 - val_loss: 0.5061 - val_accuracy: 0.8528


 [Elapsed Time: 0:04:35]  [|                       #            |] (  0.1 B/s) 


⠋ Please wait while we send the remaining data to the markov backend...⠙ Please wait while we send the remaining data to the markov backend...⠹ Please wait while we send the remaining data to the markov backend...⠸ Please wait while we send the remaining data to the markov backend...⠼ Please wait while we send the remaining data to the markov backend...⠴ Please wait while we send the remaining data to the markov backend...⠦ Please wait while we send the remaining data to the markov backend...⠧ Please wait while we send the remaining data to the markov backend...⠇ Please wait while we send the remaining data to the markov backend...⠏ Please wait while we send the remaining data to the markov backend...⠋ Please wait while we send the remaining data to the markov backend...⠙ Please wait while we send the remaining data to the markov backend...

 [Elapsed Time: 0:04:41]  [|        #                           |] (  0.4 B/s) 


✔ Recording has successfully completed.
ℹ You can view the experiment at \https://app.markovml.com/pioneer-wsp-27a2ukk2as/proj/488emc37Q5Ck4Z/experiments/hp-8T8zNMzfGSHKsFX5Pafoar8]8;;\


In [18]:
evaluation_recorder3 = markov.EvaluationRecorder(
    name=f"Testing for Best_LSTM_Model",
    model_id="FCcih5bHwMoN8XMrTR",
    project_id="vXmg4DkwjkU3JR",
    dataset_id="4Sjrh3yito3qKjujy"
)
evaluation_recorder3.register()

ℹ Creating EvaluationRecorder object Testing for Best_LSTM_Model. Call recorder_object.register() to register with MarkovML backend.
⠋ Registering evaluation recorder Testing for Best_LSTM_Model⠙ Registering evaluation recorder Testing for Best_LSTM_Model⠹ Registering evaluation recorder Testing for Best_LSTM_Model⠸ Registering evaluation recorder Testing for Best_LSTM_Model⠼ Registering evaluation recorder Testing for Best_LSTM_Model⠴ Registering evaluation recorder Testing for Best_LSTM_Model

 [Elapsed Time: 0:03:57]  [|  #                                 |] (  0.4 B/s) 

⠦ Registering evaluation recorder Testing for Best_LSTM_Model⠧ Registering evaluation recorder Testing for Best_LSTM_Model⠇ Registering evaluation recorder Testing for Best_LSTM_Model⠏ Registering evaluation recorder Testing for Best_LSTM_Model⠋ Registering evaluation recorder Testing for Best_LSTM_Model⠙ Registering evaluation recorder Testing for Best_LSTM_Model⠹ Registering evaluation recorder Testing for Best_LSTM_Model⠸ Registering evaluation recorder Testing for Best_LSTM_Model⠼ Registering evaluation recorder Testing for Best_LSTM_Model⠴ Registering evaluation recorder Testing for Best_LSTM_Model⠦ Registering evaluation recorder Testing for Best_LSTM_Model⠧ Registering evaluation recorder Testing for Best_LSTM_Model⠇ Registering evaluation recorder Testing for Best_LSTM_Model⠏ Registering evaluation recorder Testing for Best_LSTM_Model⠋ Registering evaluation recorder Testing for Best_LSTM_Model⠙ Registering evaluation recorder Testing for Best_LSTM_Model⠹ Registering evaluation

 [Elapsed Time: 0:03:56]  [|           #                        |] (  0.1 B/s) 

✔ Registering evaluation recorder Testing for Best_LSTM_Model⠴ Registering evaluation recorder Testing for Best_LSTM_Model Evaluation recorder creation for EvaluationRecording(name='Testing for Best_LSTM_Model', model_id='FCcih5bHwMoN8XMrTR', note='', dataset_id='4Sjrh3yito3qKjujy', info={'project_id': 'vXmg4DkwjkU3JR'}) successful.


 [Elapsed Time: 0:04:57]  [|                      #             |] (  0.3 B/s) 

In [19]:
y_pred = model.predict(X_test)
orig_copy = y_pred.tolist()
y_pred[y_pred > 0.5] = 1
y_pred[y_pred <= 0.5] = 0
y_pred=y_pred.astype("int32")


 [Elapsed Time: 0:04:59]  [|                            #       |] (  0.3 B/s) 

 9/42 [=====>........................] - ETA: 4s

 [Elapsed Time: 0:04:58]  [|                              #     |] (  0.1 B/s) 

10/42 [======>.......................] - ETA: 4s

 [Elapsed Time: 0:05:01]  [|        #                           |] (  0.3 B/s) 

25/42 [================>.............] - ETA: 2s

 [Elapsed Time: 0:05:03]  [|           #                        |] (  0.3 B/s) 

42/42 [==============================] - 6s 138ms/step


 [Elapsed Time: 0:05:05]  [|                                  # |] (  0.3 B/s) 

 [Elapsed Time: 0:05:08]  [|          #                         |] ( 10.3 s/B) 

In [20]:
def _get_cost(inferred, actual):
    if actual == inferred:
        return 0
    else:
        return random.randint(2, 5)
    
urid = 1
for prob, pred, orig, txt in zip(orig_copy, y_pred, y_test, X_test):
    urid=urid+1
    mi_record = SingleTagInferenceRecord(
        inferred=float(pred[0]),
        actual=float(orig),
        urid=urid,
        score=float(prob[0]),
        custom_metrics=[
            RecordCustomMetric(label="Cost", value=_get_cost(float(pred[0]), float(orig))),
            RecordCustomMetric(label="Probability", value=float(prob[0]))
        ]
    )
    evaluation_recorder3.add_record(mi_record)
outcome = evaluation_recorder3.finish()
print(outcome)

Upload Progress : 100%|████████████████████████| 2/2 [00:04<00:00,  2.25s/batch]


EvaluationRecordingFinishResponse(count=1318, recording_id='7pJzkqVjX72DA2EF2LRs3yH', return_code='OK', message='', run_id='GXL7ZEjZMuJk4LwLoj9')


 [Elapsed Time: 0:06:47]  [|                          #         |] (  0.2 B/s) 

# Testing on Data from other Social Media Sites(Reddit)

- Evaluates the best model on unseen Reddit data, presenting a classification report, confusion matrix heatmap, and ROC curve for performance assessment.

In [23]:
data = pd.read_csv('../Data/Cleaned_Reddit.csv')

unseen_text = data['cleaned_text']
unseen_labels = data['label']

unseen_sequences = tokenizer.texts_to_sequences(unseen_text)
unseen_X = pad_sequences(unseen_sequences, maxlen=max_seq_len)
unseen_y = np.array(unseen_labels)

unseen_y_pred = model.predict(unseen_X)

orig_copy = unseen_y_pred.tolist()
unseen_y_pred[unseen_y_pred > 0.5] = 1
unseen_y_pred[unseen_y_pred <= 0.5] = 0
unseen_y_pred=unseen_y_pred.astype("int32")


 [Elapsed Time: 0:07:02]  [|                         #          |] (  0.2 B/s) 

24/24 [==============================] - 3s 120ms/step


In [27]:
evaluation_recorder1 = markov.EvaluationRecorder(
    name=f"Unseen Data Testing for Best_LSTM_Model",
    model_id="FCcih5bHwMoN8XMrTR",
    project_id="vXmg4DkwjkU3JR",
    dataset_id="4Sjrh3yito3qKjujy"
)
evaluation_recorder1.register()

ℹ Creating EvaluationRecorder object Unseen Data Testing for Best_LSTM_Model. Call recorder_object.register() to register with MarkovML backend.
✔ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠙ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠹ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠸ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠼ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠴ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠦ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠧ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠇ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠏ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠋ Registering evaluation recorder Unseen Data Testing for Best_LSTM_Model⠙ Registering evaluation recorder Unseen Data Testin

 [Elapsed Time: 0:07:53]  [|                    #               |] (  0.2 B/s) 

In [25]:
def _get_cost(inferred, actual):
    if actual == inferred:
        return 0
    else:
        return random.randint(2, 5)

 [Elapsed Time: 0:07:20]  [|            #                       |] (  0.2 B/s) 

In [28]:
urid = 1
for prob, pred, orig, txt in zip(orig_copy, unseen_y_pred, unseen_y, unseen_X):
    urid = urid + 1
    mi_record = SingleTagInferenceRecord(
        inferred=float(pred[0]),
        actual=float(orig),
        urid=urid,
        score=float(prob[0]),
        custom_metrics=[
            RecordCustomMetric(label="Cost", value=_get_cost(float(pred[0]), float(orig))),
            RecordCustomMetric(label="Probability", value=float(prob[0]))
        ]
    )
    evaluation_recorder1.add_record(mi_record)
outcome = evaluation_recorder1.finish()
print(outcome)

Upload Progress : 100%|████████████████████████| 1/1 [00:01<00:00,  1.04s/batch]


EvaluationRecordingFinishResponse(count=752, recording_id='7cvE7Fx5Jh9zFJnaAitdcMH', return_code='OK', message='', run_id='cBBZBVM7q7vs4GL6Ccf')


 [Elapsed Time: 0:17:15]  [|               #                    |] ( 10.4 s/B) 